In [29]:
import requests
import pandas as pd
import json
from dateutil import parser

In [30]:
API_KEY = "40ef02d3e09569a679de4cffd812f150-d456dbb8d4d206c87a6b725a6cb847db"
ACCOUNT_ID = "101-001-25647120-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [31]:
session = requests.Session()

In [32]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [33]:
params = dict(
    count = 10,
    granularity = ["H1", "H4"],
    price = "MBA"
)

In [34]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [35]:
response = session.get(url, params=params, data=None, headers=None)

In [36]:
response.status_code

200

In [37]:
data = response.json()

In [38]:
instruments_list = data['instruments']

In [39]:
len(instruments_list)

68

In [40]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [41]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate'] 

In [42]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i }
    

In [43]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [44]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [45]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o]) 
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity=["H1", "H4"]):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No Candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    

In [46]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [47]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 10 candles, 2023-05-04 05:00:00+00:00 2023-05-05 17:00:00+00:00


In [48]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [51]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H2", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4001 candles, 2022-09-14 03:00:00+00:00 2023-05-05 20:00:00+00:00
EUR_USD H2 4001 candles, 2022-01-24 10:00:00+00:00 2023-05-05 19:00:00+00:00
EUR_USD H4 4001 candles, 2020-10-12 01:00:00+00:00 2023-05-05 17:00:00+00:00
EUR_GBP H1 4001 candles, 2022-09-14 03:00:00+00:00 2023-05-05 20:00:00+00:00
EUR_GBP H2 4001 candles, 2022-01-24 08:00:00+00:00 2023-05-05 19:00:00+00:00
EUR_GBP H4 4001 candles, 2020-10-12 01:00:00+00:00 2023-05-05 17:00:00+00:00
EUR_JPY H1 4001 candles, 2022-09-14 03:00:00+00:00 2023-05-05 20:00:00+00:00
EUR_JPY H2 4001 candles, 2022-01-24 00:00:00+00:00 2023-05-05 19:00:00+00:00
EUR_JPY H4 4001 candles, 2020-10-09 17:00:00+00:00 2023-05-05 17:00:00+00:00
EUR_CHF H1 4001 candles, 2022-09-14 03:00:00+00:00 2023-05-05 20:00:00+00:00
EUR_CHF H2 4001 candles, 2022-01-24 10:00:00+00:00 2023-05-05 19:00:00+00:00
EUR_CHF H4 4001 candles, 2020-10-12 01:00:00+00:00 2023-05-05 17:00:00+00:00
EUR_NZD H1 4001 candles, 2022-09-14 03:00:00+00:00 2023-05-05 20:00:00+00:00